<a href="https://colab.research.google.com/github/surajghuwalewala/CE888_Data_Science_and_Decision_Making/blob/master/Lab_8/1_Exercise-Food-11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Download the VGG16 file from Github
! curl https://raw.githubusercontent.com/surajghuwalewala/CE888_Data_Science_and_Decision_Making/master/Lab_8/vgg16.py --output vgg16.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8740  100  8740    0     0   193k      0 --:--:-- --:--:-- --:--:--  193k


In [2]:
!pip install tensorflow==1.3.0
!pip install keras==2.0.7

  Using cached https://files.pythonhosted.org/packages/7c/9f/57e1404fc9345759e4a732c4ab48ab4dd78fd1e60ee1270442b8850fa75f/tensorflow-1.3.0-cp36-cp36m-manylinux1_x86_64.whl
     |████████████████████████████████| 1.6MB 4.9MB/s 
     |████████████████████████████████| 890kB 49.7MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=be9adab4f00db4542d3c66ab0c81f452bce38ed93facc29bb36dcf0b60b36cbd
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
ERROR: stable-baselines 2.2.1 has requirement tensorflow>=1.5.0, but you'll have tensorflow 1.3.0 which is incompatible.
ERROR: magenta 0.3.19 has requirement tensorflow>=1.12.0, but you'll have tensorflow 1.3.0 which is incompatible.
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.0
    Uninstalling b

In [3]:
import os
os.environ['KAGGLE_USERNAME'] = "surajghuwalewala" # username from the json file
os.environ['KAGGLE_KEY'] = "c14ff4f2803c1ffb349c4b9e1a57020b" # key from the json file
DOWNLOAD_DATA = True

if DOWNLOAD_DATA:
    !kaggle datasets download -d trolukovich/food11-image-dataset # api copied from kaggle
    !unzip -q /content/food11-image-dataset.zip   ## '-q' unzips without verbose output

100% 1.08G/1.08G [00:16<00:00, 104MB/s] 
100% 1.08G/1.08G [00:16<00:00, 71.9MB/s]


In [0]:
train_dir = '/content/training/'
test_dir = '/content/evaluation/'
val_dir = '/content/validation/'

In [0]:
from keras.preprocessing.image import ImageDataGenerator

# datagen = ImageDataGenerator(rescale=1./255)

IMG_SIZE = 224

train_generator = datagen.flow_from_directory(train_dir,
                                                    target_size=(IMG_SIZE, IMG_SIZE), 
                                                    batch_size=100,
                                                    class_mode='categorical')

validation_generator = datagen.flow_from_directory(val_dir,
                                                        target_size=(IMG_SIZE, IMG_SIZE),
                                                        batch_size=100,
                                                        class_mode='categorical')

test_generator = datagen.flow_from_directory(test_dir,
                                                    target_size=(IMG_SIZE, IMG_SIZE),
                                                    batch_size=100,
                                                    class_mode='categorical')


In [0]:
# from keras import layers
# from keras import models
# from keras.layers import BatchNormalization, Activation



# model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), input_shape=(IMG_SIZE, IMG_SIZE, 3)))
# model.add(Activation('relu'))
# model.add(layers.MaxPooling2D((2, 2), padding='same'))
# model.add(layers.Dropout(0.25))

# model.add(layers.Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(layers.MaxPooling2D((2, 2), padding='same'))
# model.add(layers.Dropout(0.25))

# model.add(layers.Conv2D(256, (3, 3)))
# model.add(Activation('relu'))
# model.add(layers.MaxPooling2D((2, 2), padding='same'))
# model.add(layers.Dropout(0.25))


# model.add(layers.Flatten())
# model.add(layers.Dense(512, activation='relu'))
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dense(11, activation='softmax'))
# model.summary()

In [0]:
from keras.layers import merge, Input
from vgg16 import VGG16
from keras.layers import Dense, Activation, Flatten
from keras.models import Model


image_input = Input(shape=(224, 224, 3)) # shape of a single image

model = VGG16(input_tensor=image_input, include_top=True,weights='imagenet') # load VGG-16 model with 'top = true'
model.summary()

In [0]:
last_layer = model.get_layer('fc2').output
#x= Flatten(name='flatten')(last_layer)
out = Dense(11, activation='softmax', name='output')(last_layer)  ## 11 classes
custom_vgg_model = Model(image_input, out)
custom_vgg_model.summary()


In [0]:
for layer in custom_vgg_model.layers[:-1]:
	layer.trainable = False

custom_vgg_model.layers[3].trainable
custom_vgg_model.summary()

In [0]:
custom_vgg_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [0]:
history = custom_vgg_model.fit_generator(train_generator,
                              steps_per_epoch=30,
                              epochs=5,
                              validation_data=validation_generator,
                              validation_steps=25)

In [0]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()